# Gallery Example: M/M/k Queue (Multi-Server)

This example demonstrates an M/M/k queueing system:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Exponential service times
- **Servers**: k parallel servers (default k=3)
- **Capacity**: Infinite
- **Scheduling**: FCFS with multiple servers

Multi-server queues are fundamental models for systems with parallel service capacity.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mmk(k=3):    """Create M/M/k queueing model        Parameters:    - k: Number of servers (default 3)    """    model = Network(f'M/M/{k}')        # Block 1: nodes    source = Source(model, 'mySource')    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)    queue.set_number_of_servers(k)  # Set number of servers    sink = Sink(model, 'mySink')        # Block 2: classes    oclass = OpenClass(model, 'myClass')    source.set_arrival(oclass, Exp(2))     # λ = 2    queue.set_service(oclass, Exp(1))      # μ = 1 per server, so total μ = k        # Block 3: topology    P = model.init_routing_matrix()    P.add_route(oclass, source, queue, 1.0)    P.add_route(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the model with default k=3 serversk_servers = 3model = gallery_mmk(k_servers)print(f"Servers: {k_servers}")print(f"Arrival rate: λ = 2")print(f"Service rate per server: μ = 1")print(f"Total service capacity: {k_servers} × 1 = {k_servers}")print(f"System utilization: ρ = λ/(k×μ) = 2/{k_servers} = {2/k_servers:.3f}")

## Theoretical Analysis for M/M/k

For M/M/k with λ=2, μ=1 per server, k=3:
- **Traffic Intensity**: a = λ/μ = 2/1 = 2
- **System Utilization**: ρ = a/k = 2/3 ≈ 0.667
- **Stability**: System is stable since ρ < 1

The M/M/k model is more complex than M/M/1 due to multiple servers:
- Jobs can be served immediately if any server is free
- Queueing only occurs when all k servers are busy
- Performance significantly better than equivalent M/M/1

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Compare M/M/k with equivalent M/M/1
print("\n=== Comparison with Equivalent M/M/1 ===")

def create_equivalent_mm1():
    """Create M/M/1 with same total service capacity"""
    model_mm1 = Network('M/M/1-Equivalent')
    source = Source(model_mm1, 'Source')
    queue = Queue(model_mm1, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_mm1, 'Sink')
    
    oclass = OpenClass(model_mm1, 'Class')
    source.set_arrival(oclass, Exp(2))     # Same arrival rate
    queue.set_service(oclass, Exp(3))      # Total service rate = k×μ = 3
    
    P = model_mm1.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

model_mm1 = create_equivalent_mm1()
solver_mmk = MVA(model)
solver_mm1 = MVA(model_mm1)

avg_table_mmk = solver_mmk.avg_table()
avg_table_mm1 = solver_mm1.avg_table()

# Extract performance metrics
mmk_util = float(avg_table_mmk.iloc[1, 1])      # M/M/k utilization
mmk_resp = float(avg_table_mmk.iloc[1, 2])      # M/M/k response time
mmk_length = float(avg_table_mmk.iloc[1, 3])    # M/M/k queue length

mm1_util = float(avg_table_mm1.iloc[1, 1])      # M/M/1 utilization
mm1_resp = float(avg_table_mm1.iloc[1, 2])      # M/M/1 response time
mm1_length = float(avg_table_mm1.iloc[1, 3])    # M/M/1 queue length

print(f"Performance Comparison (same total service capacity):")
print(f"")
print(f"M/M/{k_servers} (multi-server):")
print(f"  Utilization:   {mmk_util:.4f}")
print(f"  Response Time: {mmk_resp:.4f}")
print(f"  Queue Length:  {mmk_length:.4f}")
print(f"")
print(f"M/M/1 (single fast server):")
print(f"  Utilization:   {mm1_util:.4f}")
print(f"  Response Time: {mm1_resp:.4f}")
print(f"  Queue Length:  {mm1_length:.4f}")
print(f"")
print(f"Multi-server advantage:")
print(f"  Response time: {(mm1_resp / mmk_resp):.2f}x better with multiple servers")
print(f"  Queue length:  {(mm1_length / mmk_length):.2f}x lower with multiple servers")
print(f"")
print(f"Note: Multiple servers provide better performance than a single fast server")
print(f"due to reduced waiting time variability and improved resource utilization.")

In [ ]:
# Analyze scaling with number of servers
print("\n=== Server Scaling Analysis ===")

def analyze_mmk_scaling(max_k=6):
    """Analyze performance as number of servers increases"""
    results = []
    
    for k in range(1, max_k + 1):
        model_k = gallery_mmk(k)
        solver_k = MVA(model_k)
        avg_table_k = solver_k.avg_table()
        
        util = float(avg_table_k.iloc[1, 1])
        resp_time = float(avg_table_k.iloc[1, 2])
        queue_length = float(avg_table_k.iloc[1, 3])
        
        # Theoretical utilization
        rho_theory = 2.0 / k  # λ/(k×μ) = 2/(k×1)
        
        results.append((k, rho_theory, util, resp_time, queue_length))
    
    return results

scaling_results = analyze_mmk_scaling(6)

print("k | ρ_theory | Utilization | Response Time | Queue Length | Servers Cost")
print("-" * 75)

for k, rho_theory, util, resp_time, queue_length, in scaling_results:
    servers_cost = k  # Linear cost assumption
    print(f"{k} |   {rho_theory:.3f}   |    {util:.4f}    |     {resp_time:.4f}    |    {queue_length:.4f}     |      {servers_cost}")

print(f"\nKey Insights:")
print(f"1. Utilization decreases as 1/k (diminishing returns)")
print(f"2. Response time improves significantly with additional servers")
print(f"3. Queue length decreases dramatically with more servers")
print(f"4. Cost-benefit tradeoff: server cost vs. performance improvement")

In [ ]:
# Analyze stability boundary
print("\n=== Stability Analysis ===")

def analyze_stability_boundary(k=3):
    """Analyze system behavior near stability boundary"""
    # For stability: λ < k×μ, so with μ=1, we need λ < k
    lambda_values = np.arange(0.5, k-0.1, 0.5)
    
    print(f"M/M/{k} stability analysis (μ=1 per server, capacity={k}):")
    print(f"Arrival Rate | Utilization | Response Time | Queue Length | Status")
    print("-" * 70)
    
    for lam in lambda_values:
        try:
            model_stab = Network(f'M/M/{k}-λ{lam}')
            source = Source(model_stab, 'Source')
            queue = Queue(model_stab, 'Queue', SchedStrategy.FCFS)
            queue.set_number_of_servers(k)
            sink = Sink(model_stab, 'Sink')
            
            oclass = OpenClass(model_stab, 'Class')
            source.set_arrival(oclass, Exp(lam))
            queue.set_service(oclass, Exp(1))
            
            P = model_stab.init_routing_matrix()
            P.add_route(oclass, source, queue, 1.0)
            P.add_route(oclass, queue, sink, 1.0)
            model_stab.link(P)
            
            solver_stab = MVA(model_stab)
            avg_table_stab = solver_stab.avg_table()
            
            util = float(avg_table_stab.iloc[1, 1])
            resp_time = float(avg_table_stab.iloc[1, 2])
            queue_length = float(avg_table_stab.iloc[1, 3])
            
            rho = lam / k
            status = "Stable" if rho < 1 else "Unstable"
            
            print(f"    {lam:4.1f}     |    {util:.4f}    |     {resp_time:.4f}    |    {queue_length:.4f}     | {status}")
            
        except Exception as e:
            print(f"    {lam:4.1f}     |    Error    |     Error     |    Error      | Unstable")

analyze_stability_boundary(3)

print(f"\nStability Condition: λ < k×μ")
print(f"For k=3, μ=1: λ must be < 3 for stability")
print(f"As λ approaches k×μ, response time and queue length grow rapidly.")